In [1]:
import numpy as np
import pyod
import pickle
import sklearn
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score
import scipy

# 基于Arcfacefacenet 输出的设备类型分类概率，以及对设备声纹的特征，来进行 无监督的异常声纹识别
## 未对Arcfacenet做任何后处理时，模型的异常检测分数为：
- ToyCar-->{'auc': 0.5667555555555555, 'pauc': 0.5136257309941521}
- ToyTrain-->{'auc': 0.5562611111111111, 'pauc': 0.516046783625731}
- bearing-->{'auc': 0.5404722222222221, 'pauc': 0.539812030075188}
- fan-->{'auc': 0.5892666666666666, 'pauc': 0.5390058479532164}
- gearbox-->{'auc': 0.45812777777777774, 'pauc': 0.4905847953216374}
- slider-->{'auc': 0.7778, 'pauc': 0.6098245614035088}
- valve-->{'auc': 0.5498888888888889, 'pauc': 0.5291520467836257}

- mean_AUC-->0.5769388888888889
- mean_pAUC-->0.5340073994510085


使用有监督算法的上限：
从这个结果中可以看出，embedding, score,Logits相比，包含的异常信息量：Embed>Score>Logits。
所以下一步的计划是，将集中在多分类模型下，基于Embed的异常检测识别。
ToyCar
Using Logits:0.5733333333333334
Using Embed:0.8028169014084507
Using Score:0.5970149253731343
ToyTrain
Using Logits:0.44155844155844154
Using Embed:0.5217391304347826
Using Score:0.5384615384615384
bearing
Using Logits:0.6578947368421053
Using Embed:0.8536585365853658
Using Score:0.72
fan
Using Logits:0.6756756756756757
Using Embed:0.7297297297297297
Using Score:0.6142857142857143
gearbox
Using Logits:0.6567164179104478
Using Embed:0.8289473684210527
Using Score:0.5915492957746479
slider
Using Logits:0.7662337662337663
Using Embed:0.8354430379746836
Using Score:0.6973684210526315
valve
Using Logits:0.6
Using Embed:0.5473684210526316
Using Score:0.5569620253164557



使用ECOD
ToyCar
Using Logits:0.5372340425531915
Using Embed:0.5372340425531915
ToyTrain
Using Logits:0.5132275132275133
Using Embed:0.5132275132275133
bearing
Using Logits:0.5714285714285714
Using Embed:0.5714285714285714
fan
Using Logits:0.5875
Using Embed:0.5875
gearbox
Using Logits:0.5068493150684932
Using Embed:0.5068493150684932
slider
Using Logits:0.8021390374331551
Using Embed:0.8021390374331551
valve
Using Logits:0.4861111111111111
Using Embed:0.4861111111111111




In [2]:
'''
读取已经处理好的数据(通过运行my_train训练Arcface网络。然后再运行my_test即可得到)
其中，train_data_dict和test_data_dict 按照设备类型进行划分，每个设备类型下包括：predict_logits，normal_label，embeddings
predict_logits: Arcface分类网络对设备声纹的分类结果（没有经过softmax处理的）
embeddings: Arcface网络对设备声纹的embedding（就是ArcHead 模块的Input部分）
normal_label: 设备声纹正常与否的标签，0--表示normal  1--表示abnormal
'''
f_read = open('./data/asd_train.pkl', 'rb')
train_data_dict = pickle.load(f_read)
f_read.close()

f_read = open('./data/asd_test.pkl', 'rb')
test_data_dict = pickle.load(f_read)
f_read.close()
print(train_data_dict.keys())
print(test_data_dict.keys())

dict_keys(['ToyCar', 'ToyTrain', 'bearing', 'fan', 'gearbox', 'slider', 'valve'])
dict_keys(['ToyCar', 'ToyTrain', 'bearing', 'fan', 'gearbox', 'slider', 'valve'])


## 使用异常检测算法对异常进行检测

pyod--
1. LOF: 性能与直接进行检测区别不大 0.52333（需要模型在训练时，同时将正常与异常数据一起进行建模，与我们的场景不匹配）
2. CBLOF: 本质上与LOF算法类似，实际效果不佳 0.5116666666666667（需要模型在训练时，同时将正常与异常数据一起进行建模，与我们的场景不匹配）
3. IForest:
5. OCSVM
9. ECOD: 效果最接近于有监督训练
10. KNN: 效果不佳

In [3]:
machine_type = 'gearbox'

logit_train = train_data_dict[machine_type]['predict_logits']
embed_train = train_data_dict[machine_type]['embeddings']

logit_test = test_data_dict[machine_type]['predict_logits']
embed_test = test_data_dict[machine_type]['embeddings']
normal_label_test_gt = test_data_dict[machine_type]['normal_label']

In [ ]:
#GMM


In [97]:
#from pyod.models.lof import LOF
#ab_clf = LOF(n_neighbors=5, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, contamination=0.01, n_jobs=1, novelty=True)

#from pyod.models.cblof import CBLOF
#ab_clf =  CBLOF(n_clusters=8, contamination=0.1, clustering_estimator=None, alpha=0.9, beta=5, use_weights=False, check_estimator=False, n_jobs=1)

# from pyod.models.iforest import IForest
# ab_clf = IForest(n_estimators=100, max_samples='auto', contamination=0.1, max_features=1.0, bootstrap=False, n_jobs=1, behaviour='old', random_state=None, verbose=0)

# from pyod.models.ecod import ECOD
# ab_clf1 = ECOD(contamination=0.1, n_jobs=1)
# ab_clf2 = ECOD(contamination=0.1, n_jobs=1)

# from pyod.models.knn import KNN
# ab_clf = KNN(contamination=0.1, n_neighbors=5, method='largest', radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=1)

from pyod.models.ocsvm import OCSVM
ab_clf1 = OCSVM(kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, nu=0.5, shrinking=True, cache_size=200, verbose=False, max_iter=-1, contamination=0.1)
ab_clf2 = OCSVM(kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, nu=0.5, shrinking=True, cache_size=200, verbose=False, max_iter=-1, contamination=0.1)

In [98]:
ab_clf1.fit(np.vstack([logit_train]))
y_pred = ab_clf1.predict(logit_test)
print("Using Logits:")
print(accuracy_score(normal_label_test_gt, y_pred))

ab_clf2.fit(np.vstack([embed_train]))
y_pred = ab_clf2.predict(embed_test)
print("Using Embed:")
print(accuracy_score(normal_label_test_gt, y_pred))

Using Logits:
0.47833333333333333
Using Embed:
0.5033333333333333


In [107]:
scipy.special.softmax(logit_test,axis=1)


array([[1.02717879e-10, 3.65052406e-08, 3.12133710e-09, ...,
        9.99999960e-01, 2.24390277e-11, 3.36198858e-10],
       [1.38314268e-09, 7.67856612e-07, 3.42278471e-08, ...,
        9.99999082e-01, 1.17995127e-11, 1.14709232e-07],
       [8.19233843e-10, 7.13227062e-07, 1.47231117e-08, ...,
        9.99999264e-01, 3.70883542e-11, 7.30584048e-09],
       ...,
       [9.94984951e-11, 2.57717113e-08, 2.32285015e-08, ...,
        9.99999947e-01, 1.25907429e-12, 3.73242970e-09],
       [2.05429011e-09, 6.67020378e-09, 9.00851602e-09, ...,
        9.99999981e-01, 1.56393579e-13, 1.35341958e-09],
       [1.26582565e-09, 2.46259889e-08, 1.50778336e-08, ...,
        9.99999959e-01, 2.38331567e-12, 1.16878693e-10]])

In [117]:
from pyod.models.ecod import ECOD
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
keys = test_data_dict.keys()
for machine_type in keys:
    logit_train = train_data_dict[machine_type]['predict_logits']
    embed_train = train_data_dict[machine_type]['embeddings']

    logit_test = test_data_dict[machine_type]['predict_logits']
    embed_test = test_data_dict[machine_type]['embeddings']
    normal_label_test_gt = test_data_dict[machine_type]['normal_label']

    # ab_clf1 = ECOD(contamination=0.1, n_jobs=1)
    # ab_clf2 = ECOD(contamination=0.1, n_jobs=1)
    ab_clf1 = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=1, objective='binary:logistic')
    ab_clf2 = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=1, objective='binary:logistic')

    x_train1, x_test1, y_train1, y_test1 = train_test_split(logit_test, normal_label_test_gt, test_size = 0.25)
    ab_clf1.fit(x_train1, y_train1)
    y_pred1 = ab_clf1.predict(x_test1)

    x_train2, x_test2, y_train2, y_test2 = train_test_split(embed_test, normal_label_test_gt, test_size = 0.25)
    ab_clf2.fit(x_train2, y_train2)
    y_pred2 = ab_clf2.predict(x_test2)
    print(machine_type)
    print("Using Logits:" + str(precision_score(y_test1, y_pred1, pos_label=1)))
    print("Using Embed:" + str(precision_score(y_test2, y_pred2, pos_label=1)))

    ab_clf3 = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=1, objective='binary:logistic')
    x_train3, x_test3, y_train3, y_test3 = train_test_split(scipy.special.softmax(logit_test, axis=1), normal_label_test_gt, test_size = 0.25)
    ab_clf3.fit(x_train3, y_train3)
    y_pred3 = ab_clf3.predict(x_test3)
    print("Using Score:" + str(precision_score(y_test3, y_pred3, pos_label=1)))


ToyCar
Using Logits:0.5733333333333334
Using Embed:0.8028169014084507
Using Score:0.5970149253731343
ToyTrain
Using Logits:0.44155844155844154
Using Embed:0.5217391304347826
Using Score:0.5384615384615384
bearing
Using Logits:0.6578947368421053
Using Embed:0.8536585365853658
Using Score:0.72
fan
Using Logits:0.6756756756756757
Using Embed:0.7297297297297297
Using Score:0.6142857142857143
gearbox
Using Logits:0.6567164179104478
Using Embed:0.8289473684210527
Using Score:0.5915492957746479
slider
Using Logits:0.7662337662337663
Using Embed:0.8354430379746836
Using Score:0.6973684210526315
valve
Using Logits:0.6
Using Embed:0.5473684210526316
Using Score:0.5569620253164557
